In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
import json
import re

with open('datasets/data.json') as file:
    data = json.loads(file.read())

corpus: list[str] = [entry['summary'] for entry in data]

split_value = int(0.9 * len(corpus))

train_data: list[dict[str, str]] = corpus[:split_value]
test_data: list[dict[str, str]] = corpus[split_value:]

In [ ]:
from language_model import generation

model = generation.LanguageModel(device=device)

print(sum([p.numel() for p in model.encoder.parameters()]) / 1e6, 'M parameters')

In [ ]:
model.predict(' ')

In [ ]:
from language_model import train

trainer = train.ModelTrainer(model, train_data, test_data)

trainer.train()

In [ ]:
model.predict(' ')

In [ ]:
model.predict('Deep Learning')

In [ ]:
model.save('model.pt')